In [1]:
import geopandas as gpd
from dem_stitcher.geojson_io import read_geojson_gzip
import pandas as pd
# import folium
import leafmap
from tqdm import tqdm

#### Read data overview tables

In [2]:
# Read burst summary table generated in dist-s1-research/marshak/4_rtc_organization/0_Organize-RTC-Data.ipynb
df_burst_summary = read_geojson_gzip('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/marshak/4_rtc_organization/burst_extent_subset.geojson.zip')
df_burst_summary.head()

,geometry,jpl_burst_id,orbit_pass,count,is_val_burst,track_number
0,"POLYGON ((-97.90103 16.86358, -97.06559 17.014...",T005-008688-IW2,ASCENDING,66,False,5
1,"POLYGON ((-97.09889 17.06678, -96.35903 17.196...",T005-008688-IW3,ASCENDING,66,False,5
2,"POLYGON ((-98.69437 16.82087, -97.89903 16.966...",T005-008689-IW1,ASCENDING,66,False,5
3,"POLYGON ((-97.93485 17.03023, -97.09864 17.180...",T005-008689-IW2,ASCENDING,66,False,5
4,"POLYGON ((-97.13196 17.23328, -96.39139 17.362...",T005-008689-IW3,ASCENDING,66,False,5


In [3]:
# Read rtc information table generated in dist-s1-research/marshak/4_rtc_organization/0_Organize-RTC-Data.ipynb
df_rtc = pd.read_json('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/marshak/4_rtc_organization/rtc_s1_table.json.zip')
df_rtc.head()

,rtc_s1_id,input_slc_id,jpl_burst_id,bursts_per_slc_input,rtc_s1_vv_url,rtc_s1_vh_url,rtc_s1_h5_url,acq_datetime
0,OPERA_L2_RTC-S1_T005-008688-IW2_20201010T00400...,S1A_IW_SLC__1SDV_20201010T004001_20201010T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-10 00:40:01
1,OPERA_L2_RTC-S1_T005-008688-IW2_20201022T00400...,S1A_IW_SLC__1SDV_20201022T004000_20201022T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-22 00:40:00
2,OPERA_L2_RTC-S1_T005-008688-IW2_20201103T00400...,S1A_IW_SLC__1SDV_20201103T004000_20201103T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-03 00:40:00
3,OPERA_L2_RTC-S1_T005-008688-IW2_20201115T00400...,S1A_IW_SLC__1SDV_20201115T004000_20201115T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-15 00:40:00
4,OPERA_L2_RTC-S1_T005-008688-IW2_20201127T00400...,S1A_IW_SLC__1SDV_20201127T004000_20201127T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-27 00:40:00


In [4]:
# Read validation sites from table generated in dist-s1-research/marshak/3_dist_sites/dist_hls_validation_table.ipynb
df_val_sites = gpd.read_file('/Users/cabrera/Documents/jpl_projects/opera_dist/dist-s1-research/oliver/rtc_analysis/dist_hls_val_sites.geojson')
df_val_sites_subset = gpd.sjoin(df_val_sites, df_burst_summary[['jpl_burst_id', 'geometry']], predicate='intersects', how='inner').drop(columns=['index_right']).reset_index(drop=True)
df_val_sites_subset.head()

,site_id,change_label,change_time,last_observation_time,geometry,jpl_burst_id
0,4,VLmaj,2021-11-07,2021-11-02,POINT (19.33289 4.57490),T036-076228-IW3
1,13,VLmaj,2021-10-03,NaT,POINT (48.77698 31.75490),T108-230715-IW2
2,13,VLmaj,2021-10-03,NaT,POINT (48.77698 31.75490),T101-214979-IW1
3,22,VLmin,2021-12-19,2021-12-12,POINT (36.61178 35.52891),T014-028130-IW2
4,22,VLmin,2021-12-19,2021-12-12,POINT (36.61178 35.52891),T021-043823-IW2


In [5]:
# get the unique bursts ID's
track_numbers = df_burst_summary.track_number.unique()
len(track_numbers)

58

In [6]:
# Add track numbers if they are blocking tracks with data you require
TRACKS_TO_EXCLUDE = [137, 70]

gdf = df_burst_summary.copy()
gdf.geometry = df_burst_summary.geometry.exterior

def f(feature):
    return {
        'fillColor': 'yellow' if (feature['properties']['is_val_burst']) else 'blue',
        'weight': 2,
        'fillOpacity': 0.4 * float(feature['properties']['count']) / 100
    }

m = leafmap.Map()
m.add_gdf(df_burst_summary[~df_burst_summary.track_number.isin(TRACKS_TO_EXCLUDE)].copy(), 
          layer_name=f"Burst Data", info_mode='on_click', 
          style_callback=f)
m.add_gdf(df_val_sites_subset, 
      layer_name=f"Val. Sites", info_mode='on_click')
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…